<a href="https://colab.research.google.com/github/juancamilo9/predicciones-ventas/blob/master/base_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# montaremos os datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# importamos las librerias
# importamos las librerias
# imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')


In [18]:
# omntar datos
filename = '/content/drive/MyDrive/sales_predictions.csv'
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


Hay unos valores que se deben de manejar antes e reaizar la separacón y tratamiento adecuado de los datos, como por ejemplo la redundancia de registros en una sola columna que puden signicar o hacer referencia a una misma categoría, vamos a darle un manejo a este tipo de datos antes de continuar con los demas tratamientos de los datos


En la columna Outlet_size, hay una cantodad de datos faltantes, pero a su vez en una categoría ordinal, por lo cual, vamos a convertir el tipo de objeto a int y luego mas adelante, trataremos de llenar los valores con un imputer

In [19]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [20]:
dicctionary_replace  = {'Small':0,'Medium':1,'High':2}

df['Outlet_Size'].replace(dicctionary_replace,inplace=True)


In [29]:
df['Outlet_Size'].value_counts()

1.0    2793
0.0    2388
2.0     932
Name: Outlet_Size, dtype: int64

El campo Item_Fat_Content, maneja datos categoricos, pero hay categorías repetizas, vamos a reemplazarlas y unificarlas, no trataremos su escalamietno ya que usaremos OneHotEncoder para este


In [24]:
# Tratemos esta inconsistencia
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [25]:
# Remplazo 'LF' por 'Low Fat'
df['Item_Fat_Content'].replace('LF','Low Fat',inplace=True)
# Remplazo 'low fat' por 'Low Fat'
df['Item_Fat_Content'].replace('low fat','Low Fat',inplace=True)
# Remplazo 'reg' por 'Regular'
df['Item_Fat_Content'].replace('reg','Regular',inplace=True)
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

Otro campo el cual debemos manejar antes de la separación es Outlet_Location_Type

In [35]:
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

Son datos ordinales, por lo cual, es conveniente trataros como tal para que sean analizados

In [36]:
dicctionary_replace  = {'Tier 1':0,'Tier 2':1,'Tier 3':2}

df['Outlet_Location_Type'].replace(dicctionary_replace,inplace=True)

In [37]:
df['Outlet_Location_Type'].value_counts()

2    3350
1    2785
0    2388
Name: Outlet_Location_Type, dtype: int64

In [38]:
df['Outlet_Type'].replace('Supermarket Type1','Supermarket',inplace=True)
df['Outlet_Type'].replace('Supermarket Type2','Supermarket',inplace=True)
df['Outlet_Type'].replace('Supermarket Type3','Supermarket',inplace=True)
df['Outlet_Type'].value_counts()

Supermarket      7440
Grocery Store    1083
Name: Outlet_Type, dtype: int64

# Asignación de vaoleres objetivos

In [39]:
y = df['Item_Outlet_Sales']
X = df.drop(columns='Item_Outlet_Sales')
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [40]:
#creamos los selectores para los diferentes tipos de datos
cat_selector = make_column_selector(dtype_include = 'object')
num_selector = make_column_selector(dtype_include = 'number')

In [41]:
# instnciaremos los transformadores:  most_frequent y median,
freq_imputer = SimpleImputer(strategy='most_frequent')
median_imputer = SimpleImputer(strategy='median')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)


In [42]:
# instanciamos los piplines 
# Numeric pipeline
numeric_pipe = make_pipeline(median_imputer, scaler)
numeric_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [43]:
# pipelines para los datos de tipo object
categorical_pipe = make_pipeline(freq_imputer,ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [44]:
# instanciamos los transformadores para los datos
num_tuple = (numeric_pipe,num_selector)
cat_tuple = (categorical_pipe,cat_selector)
# cReamos el column tyransformer
# ColumnTransformer
preprocessor = make_column_transformer(num_tuple, cat_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa1fccd0310>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa1fccd09d0>)])

In [45]:
# entrenamos los datos
# fit on train
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa1fccd0310>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa1fccd09d0>)])

In [46]:
# transformamos los datos
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [50]:
# comprobamos los valores
# Comprueben los valores faltantes y que los datos se escalen y tengan una codificación one-hot
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)
print('\n')
X_train_processed


0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (6392, 1586)




array([[ 0.82748547, -0.71277507,  1.82810922, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.56664432, -1.29105225,  0.60336888, ...,  0.        ,
         0.        ,  1.        ],
       [-0.12102782,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  1.        ],
       ...,
       [ 1.12389588, -0.92052713,  1.52302674, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.77599877, -0.2277552 , -0.38377708, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.82748547, -0.95867683, -0.73836105, ...,  0.        ,
         0.        ,  1.        ]])